# Topic Modeling using LDA

In [1]:
# # To be run only once
# if 0 == 1:
#     !pip install gensim
#     !pip install PyLDAvis
#     !pip install spacy
#     !python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt

import re
import spacy
import tqdm

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora

from nltk.corpus import stopwords
import nltk
import en_core_web_sm

from pprint import pprint

import pyLDAvis.gensim
import pickle 
import pyLDAvis

import time
from collections import Counter
import random

In [3]:
# Cache stop_words into hash
stop_words = stopwords.words('english')
stop_words.extend(['from'])
stop_words = Counter(stop_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

random.seed(3)

In [4]:
# nltk.download('stopwords')

## Preprocessing data

In [5]:
start = time.time()

In [ ]:
# one_drive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"
one_drive_path = "C:/Users/asaid/The Estée Lauder Companies Inc/TeamAnis - General/"

In [6]:
df = pd.read_pickle('reviews.pkl')

In [14]:
df = df[df['geography']=='USA']

In [22]:
cols = ['onlinepost_id', 'source_product_identifier', 'onlinestatement_id',
       'date', 'title', 'description', 'geography', 'channel', 'product_id',
       'rating', 'sentiment']

In [23]:
df = df[cols]

In [15]:
df.drop(columns=['geography'], inplace=True)

C:\Users\asaid\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [24]:
df.to_pickle('reviews_columns.pickle')

,onlinepost_id,source_product_identifier,onlinestatement_id,date,title,description,channel,product_id,rating,sentiment
0,OnlinePost_20191027_184110681,pimprod2006667,OnlineStatement_20191027_184110681_5,2019-09-06,Nice!,I am very pleased!,Ulta,Product_20191016_5443258,4.0,Positive
1,OnlinePost_20191027_184748557,pimprod2006667,OnlineStatement_20191027_184748557_5,2019-09-07,. I would recommended if in budget.,However I wish it got rid of the powdery look ...,Ulta,Product_20191016_5443258,4.0,Positive
2,OnlinePost_20191027_184112758,pimprod2006667,OnlineStatement_20191027_184112758_1,2019-09-13,Love it!,Love it!,Ulta,Product_20191016_5443258,5.0,Positive
3,OnlinePost_20191027_183920306,pimprod2006667,OnlineStatement_20191027_183920306_4,2019-09-27,Perfect Product!,It helps my makeup lady all day!,Ulta,Product_20191016_5443258,5.0,Positive
4,OnlinePost_20191027_184386195,pimprod2006667,OnlineStatement_20191027_184386195_1,2019-09-19,AmaIng!,AmaIng!,Ulta,Product_20191016_5443258,5.0,Neutral
...,...,...,...,...,...,...,...,...,...,...
20129659,OnlinePost_20200524_6edee6cd-4283-497c-b37f-94...,10486034,OnlineStatement_20200524_6edee6cd-4283-497c-b3...,2020-04-10,NaN,Within two days I noticed how much better my s...,Macy's,Product_20200416_18804683,5.0,Positive
20129660,OnlinePost_20200524_1c5b762d-bd4f-4935-8559-be...,2717198,OnlineStatement_20200524_1c5b762d-bd4f-4935-85...,2020-04-09,Ok,"It's a very thick cream, which I don't usually...",Bloomingdales,Product_20200416_18796718,3.0,Positive
20129662,OnlinePost_20200524_38ab5dd5-5c1d-4b7f-b2e3-6e...,3438423,OnlineStatement_20200524_38ab5dd5-5c1d-4b7f-b2...,2020-04-18,Great Product,Great Product.,Bloomingdales,Product_20200416_18799806,5.0,Positive
20129663,OnlinePost_20200524_a21bc2f7-d2b2-4660-bd9a-07...,4900069,OnlineStatement_20200524_a21bc2f7-d2b2-4660-bd...,2020-04-08,Amazingly subtle colour and radiance,So easy to use and love the fact I can still u...,Nordstrom,Product_20200416_18822798,5.0,Positive


In [ ]:
df.groupby()

In [ ]:
# df = df.sample(100, random_state=3)
# del df

In [ ]:
# Cache stop_words into hash
stop_words = stopwords.words('english')
stop_words.extend(['from'])
stop_words = Counter(stop_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def preprocess(sentences, stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    # lower case
    # Remove stop words
    # Lemmatize
    for sentence in sentences:
        doc = nlp(' '.join([token for token in gensim.utils.simple_preprocess(str(sentence), deacc=True) if token not in stop_words]) )
        yield([token.lemma_ for token in doc if token.pos_ in allowed_postags])

In [ ]:
docs = list(tqdm.tqdm(preprocess(df.values.tolist(), stop_words), position=0, leave=True))

In [ ]:
len(docs)

In [ ]:
docs = pickle.load(open("docs.p", "rb"))

In [ ]:
d = corpora.Dictionary(docs)

In [ ]:
freq = pd.DataFrame(d.dfs.values(), index=d.dfs.keys(), columns=['freq'])
freq.index.name = 'idx'
freq = freq.reset_index()
freq['token'] = freq['idx'].apply(lambda x:d[x])
freq = freq.sort_values(by='freq', ascending=False)

In [ ]:
d.token2id['transition']

In [ ]:
d.dfs[115]

In [ ]:
freq.iloc[0:50].plot.bar(x='token', y='freq', figsize=(20,20))
plt.grid()

In [ ]:
freq

In [ ]:
list(d.dfs.keys())

In [ ]:
d.id2token[112]

In [ ]:
for idx, freq in d.dfs.items():
    

In [ ]:
pd.DataFrame(d.dfs, index=d.dfs.keys())

## Validating Model

In [ ]:
# Inputs
docs = random.sample(docs, 100000)

# Create Dictionary
id2word = corpora.Dictionary(docs)
pickle.dump(id2word, open( "id2word.p", "wb" ) )

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, text, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
grid = {}
grid['Validation_Set'] = {}

nb_words = len(id2word)

# Topics range
min_topics = 6
max_topics = 10
step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
topics_range = [8]
# Alpha parameter
# Added in the loop

# Alpha
alpha = [
#     0.1, 
#     'symmetric',
#     'asymmetric'
]

# Beta parameter
beta = [
    0.1, 
#     200/nb_words
]
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.05), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
    corpus
]

corpus_title = [
#     '25% Corpus',
#     '50% Corpus',
#     '75% Corpus',
    '100% Corpus'
]

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*(len(alpha)+1)*len(topics_range)*len(corpus_title)))
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            alpha.append(50/k)
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, text=docs,
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    pbar.update(1)
    res = pd.DataFrame(model_results)
    res = pd.DataFrame(model_results).sort_values("Coherence", ascending=False)
    res.to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

In [ ]:
res

## Training Best Model

In [ ]:
best_param = res.iloc[0]
num_topics = best_param['Topics']
alpha = best_param['Alpha']
eta = best_param['Beta']

In [ ]:
# Build LDA model
int_start=time.time()
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       alpha = alpha,
                                       eta = eta,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

In [ ]:
lda_model.save('lda_test.model')

In [ ]:
print(time.time()-start)

## Predictions

In [ ]:
# df = pd.read_pickle('reviews_concat.pkl')

In [ ]:
# df = df.loc['2019']

In [ ]:
# print(len(df))

In [ ]:
# docs = list(tqdm.tqdm(preprocess(df.values.tolist(), stop_words), position=0, leave=True))

In [ ]:
# pickle.dump(docs, open( "docs.p", "wb" ) )

In [ ]:
docs = pickle.load(open("docs.p", "rb"))
id2word = pickle.load(open("id2word.p", "rb"))

In [ ]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

In [ ]:
# lda_model = gen?sim.models.LdaModel.load('lda.model')

In [ ]:
output = pd.concat([df.to_frame('description').reset_index(), pd.DataFrame(gensim.matutils.corpus2csc(lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,num_topics+1)])], axis=1, ignore_index=True)

In [ ]:
pd.DataFrame(gensim.matutils.corpus2csc(lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,num_topics+1)])

In [ ]:
df.to_frame('description').reset_index()

In [ ]:
output.to_pickle('reviews_w_topics_test.pkl')

## Appendix

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=150) # higher threshold fewer phrases.
# # trigram = gensim.models.Phrases(bigram[data_words], threshold=150)

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# # trigram_mod = gensim.models.phrases.Phraser(trigram)

# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

# # Remove Stop Words
# data_words = remove_stopwords(data_words)

# # Do lemmatization keeping only noun, adj, vb, adv
# data_words = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# # Form Bigrams
# data_words = make_bigrams(data_words)

In [ ]:
# # Build LDA model
# lda_model = gensim.models.LdaMulticore(corpus=corpus,
#                                        id2word=id2word,
#                                        num_topics=8, 
#                                        random_state=100,
#                                        chunksize=100,
#                                        passes=10,
#                                        per_word_topics=True)

In [ ]:
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [ ]:
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('Coherence Score: ', coherence_lda)